# Stargazer Examples

In [1]:
import pandas as pd
from sklearn import datasets
import statsmodels.api as sm
from stargazer.stargazer import Stargazer, LineLocation

In [2]:
diabetes = datasets.load_diabetes()
df = pd.DataFrame(diabetes.data)
df.columns = ['Age', 'Sex', 'BMI', 'ABP', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']
df['target'] = diabetes.target

est = sm.OLS(endog=df['target'], exog=sm.add_constant(df[df.columns[0:4]])).fit()
est2 = sm.OLS(endog=df['target'], exog=sm.add_constant(df[df.columns[0:6]])).fit()


stargazer = Stargazer([est, est2])

## Standard

In [3]:
# Equivalent to:
# from IPython.core.display import HTML
# HTML(stargazer.render_html())

stargazer

In [4]:
print(stargazer.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{2}{c}{\textit{Dependent variable: target}} \
\cr \cline{2-3}
\\[-1.8ex] & (1) & (2) \\
\hline \\[-1.8ex]
 ABP & 416.673$^{***}$ & 397.581$^{***}$ \\
& (69.495) & (70.870) \\
 Age & 37.241$^{}$ & 24.703$^{}$ \\
& (64.117) & (65.411) \\
 BMI & 787.182$^{***}$ & 789.744$^{***}$ \\
& (65.424) & (66.887) \\
 S1 & & 197.848$^{}$ \\
& & (143.812) \\
 S2 & & -169.243$^{}$ \\
& & (142.744) \\
 Sex & -106.576$^{*}$ & -82.862$^{}$ \\
& (62.125) & (64.851) \\
 const & 152.133$^{***}$ & 152.133$^{***}$ \\
& (2.853) & (2.853) \\
\hline \\[-1.8ex]
 Observations & 442 & 442 \\
 $R^2$ & 0.400 & 0.403 \\
 Adjusted $R^2$ & 0.395 & 0.395 \\
 Residual Std. Error & 59.976 (df=437) & 59.982 (df=435) \\
 F Statistic & 72.913$^{***}$ (df=4; 437) & 48.915$^{***}$ (df=6; 435) \\
\hline
\hline \\[-1.8ex]
\textit{Note:} & \multicolumn{2}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\


## Custom Title

In [5]:
stargazer.title('Diabetes Study')
stargazer

## Custom Model Names

In [6]:
stargazer.custom_columns(['Model 1', 'Model 2'], [1, 1])
stargazer

In [7]:
stargazer.custom_columns('Test model name')
stargazer

## Remove Model Numbers

In [8]:
stargazer.show_model_numbers(False)
stargazer

## Specify Significant Digits

In [9]:
stargazer.significant_digits(2)
stargazer

## Show Confidence Intervals Over Standard Errors

In [10]:
stargazer.show_confidence_intervals(True)
stargazer

## Subset Covariates and/or Choose Covariate Order

In [11]:
stargazer.covariate_order(['BMI', 'Age', 'S1', 'Sex'])
stargazer

## Rename Covariates

In [12]:
stargazer.rename_covariates({'Age': 'Oldness'})
stargazer

... with a function

In [13]:
stargazer.rename_covariates(lambda x : 'Years lived' if x == 'Age' else x
                           )
stargazer

... with format-specific versions:

In [14]:
from stargazer.stargazer import Label

In [15]:
stargazer.rename_covariates({
                             'S1' : Label({'html' : 'S<sub>1</sub>',
                                           'LaTeX' : 'S_1'})
                            }
                           )
stargazer

## Hide/Show Degrees of Freedom

In [16]:
stargazer.show_degrees_of_freedom(False)
stargazer

## Show Custom Notes

In [17]:
stargazer.add_custom_notes(['First note', 'Second note'])
stargazer

## Add Custom Lines

In [18]:
stargazer.add_line('More controls', ['No', 'No'])
stargazer.add_line('Robust', ['No', 'Yes'], LineLocation.HEADER_BOTTOM)
stargazer.add_line('Preferred', ['No', 'Yes'], LineLocation.FOOTER_TOP)
# Use a format-specific Label:
stargazer.add_line(Label({'html' : 'Largest R<sup>2</sup>',
                          'LaTeX' : 'Largest $R^2$'}),
                   ['Yes', 'Yes'],
                   LineLocation.FOOTER_BOTTOM
                  )
stargazer

## Change Statistical Significance Cutoffs

In [19]:
stargazer.significance_levels([0.1, 0.05, 0.07])
stargazer

## Don't Show Statistical Significance Levels In Notes

In [20]:
stargazer.append_notes(False)
stargazer

## Add spacing between rows

In [21]:
stargazer.cov_spacing = 2
stargazer

## Display panel results from linearmodels

In [26]:
from linearmodels.panel import PanelOLS, RandomEffects, PooledOLS
from linearmodels.datasets import wage_panel

dfp = wage_panel.load().set_index(["nr", "year"])

pest = PooledOLS.from_formula('lwage  ~ 1 + black + hisp + exper + expersq + married + educ + union + TimeEffects', data=dfp).fit()
rest = RandomEffects.from_formula('lwage  ~ 1 + black + hisp + exper + expersq + married + educ + union + TimeEffects', data=dfp).fit()
fest = PanelOLS.from_formula('lwage  ~ 1 + expersq + union + married + TimeEffects + EntityEffects', data=dfp).fit()

models=[pest, rest, fest]
panel_res = Stargazer(models)
panel_res.custom_columns(['Pooled', 'Random Effects', 'Fixed Effects'])
panel_res.covariate_order(['black', 'hisp', 'exper', 'expersq', 'married', 'educ', 'union', 'Intercept'])
panel_res.add_line('Time Effects', ['Yes', 'Yes', 'Yes'])
panel_res.add_line('Fixed Effects', ['No', 'No', 'Yes'])

# Add model specific statistics
panel_res.add_line('Number of indiv.', [int(m.entity_info.total) for m in models])
panel_res.add_line(Label({'html' : 'Within R<sup>2</sup>',
                          'LaTeX' : 'Within $R^2$'}),
                   [round(m.corr_squared_within,3) for m in models],
                   LineLocation.FOOTER_BOTTOM
                  )

  
panel_res